<a href="https://colab.research.google.com/github/liuhuiaren0524/GoogleColab-Github/blob/main/fuc_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/Colab Notebooks')

TensorFlow 1.x selected.
1.15.2
Mounted at /content/drive


In [4]:
import os
os.chdir("/home/lhr_rw/test")
from EVPortalToolFunction import loadHiveData

def getNissianData(stat_date):
  sql = """
  select 
    totalelectricity/10, id, collecttime
  from 
    table
  where 
    custommsgid is not null and 
    etl_date = '{}' and 
    custommsgtype = 2 and
    collecttime > 0
  sort by 
    vin, collecttime""".format(stat_date)
  col = ['totalelectricity', 'id', 'collecttime']
  return loadHiveData(sql, col)


import datetime

def getEveryDay(begin_date,end_date):
    date_list = []
    begin_date = datetime.datetime.strptime(begin_date, "%Y-%m-%d")
    end_date = datetime.datetime.strptime(end_date,"%Y-%m-%d")
    while begin_date <= end_date:
        date_str = begin_date.strftime("%Y-%m-%d")
        date_list.append(date_str)
        begin_date += datetime.timedelta(days=1)
    return date_list

begin_date = '2021-09-10'
end_date   = '2021-09-12'

Date_list = getEveryDay(begin_date, end_date)

for i in range(len(Date_list)):
  print(i)
  stat_dates = Date_list[i]
  #print(stat_date)
  #stat_date = '20210912'
  a1 = getNissianData(stat_dates)
  a1['collecttime_shift'] = a1.groupby('id')['collecttime'].shift(1)
  a1['diff'] = a1['collecttime']- a1['collecttime_shift'] 
  a1 = a1.fillna(10)
  a1['1'] = a1['totalelectricity']*a1['diff']/3600
  a1 = a1[a1['1'] > 0]
  a2 = a1.groupby('id').agg({'1':'sum'})
  print(a2.head())
  a2.to_csv('{}_data.csv'.format(stat_dates))

FileNotFoundError: ignored